In [2]:
from datapipeline import pipeline
import pandas as pd
import numpy as np
import re

ModuleNotFoundError: No module named 'datapipeline'

Load in data using our pipeline

In [1]:
pipe = pipeline()
df = pipe.load_csv('posts_2019_all.csv')
df.rename(columns ={'link_flair_css_class': 'type'}, inplace = True)

NameError: name 'pipeline' is not defined

In [21]:
print(df.shape)
df.head(100)

(236758, 10)


,created_utc,title,subreddit,author,num_comments,score,id,type,author_flair_css_class,original_title
0,1546300891,Malik Monk with a FEROCIOUS Dunk! Announcer is...,nba,exc99,64,736,abcv10,highlights,Pistons1,Malik Monk with a FEROCIOUS Dunk! Announcer is...
1,1546301066,Spurs' Gay out vs. Celtics with injured wrist,nba,bornfromblue,12,11,abcvzx,None,NaN,Spurs' Gay out vs. Celtics with injured wrist
2,1546301178,Kemba Dances Around on The Hardwood!,nba,Kraze_F35,11,35,abcwm2,highlights,ChaHornets2,Kemba Dances Around on The Hardwood!
3,1546301267,What should a player have to do for an organis...,nba,Tr3ywayy,19,0,abcx3a,None,NaN,What should a player have to do for an organis...
4,1546301353,Malik Monk beats the buzzer going into halftime!,nba,Kraze_F35,3,31,abcxmd,highlights,ChaHornets2,Malik Monk beats the buzzer going into halftime!
...,...,...,...,...,...,...,...,...,...,...
95,1546308784,The San Antonio Spurs (21-17) defeat the Bosto...,nba,Jayveesac,436,1326,abdywy,postgamethread,Lakers1,[Post Game Thread] The San Antonio Spurs (21-1...
96,1546308836,Steven Adams is the most interesting man,nba,B4IFURU-18,0,1,abdz5c,None,NaN,Steven Adams is the most interesting man
97,1546308840,"With less than 4 hours left in 2018, the San A...",nba,[deleted],0,5,abdz5s,None,NaN,"With less than 4 hours left in 2018, the San A..."
98,1546308911,[MacMahon]:James Harden’s 25 free throws attem...,nba,bondom3417,11,10,abdzhi,news,NBA,[MacMahon]:James Harden’s 25 free throws attem...


In [28]:
df['type'].replace(np.nan, 'None', inplace = True)
post_types = df['type'].unique()
post_types

array(['highlights', 'None', 'gamethread', 'postgamethread', 'news',
       'discussion', 'miscmedia', 'rostermoves', 'index', 'spoilers',
       'rnba', 'ama', 'trashtalk', 'shitpost', 'meta', 'paywall'],
      dtype=object)

In [29]:
for type in post_types:
    n = df[df['type'] == type].size
    print('Type: {0:15}  number of posts: {1}'.format(type, n))

Type: highlights       number of posts: 151290
Type: None             number of posts: 1843542
Type: gamethread       number of posts: 9135
Type: postgamethread   number of posts: 11169
Type: news             number of posts: 61110
Type: discussion       number of posts: 12834
Type: miscmedia        number of posts: 7623
Type: rostermoves      number of posts: 20403
Type: index            number of posts: 1773
Type: spoilers         number of posts: 10341
Type: rnba             number of posts: 792
Type: ama              number of posts: 45
Type: trashtalk        number of posts: 45
Type: shitpost         number of posts: 9
Type: meta             number of posts: 9
Type: paywall          number of posts: 702


In [30]:
df['title'][df['type'] == 'highlights'].tail(30)


236534    [Highlight] Jokic with the amazing tap pass to...
236541    Steve Francise Mic'd Up Highlights 2001 Rookie...
236542    [Highlight] Josh Okogie hits the contested tri...
236545    Rod Strickland with a smooth behind-the-back s...
236546                           Jokic highlights vs Angola
236548    [Highlight] Josh Okogie with the beautiful fee...
236552    Dell Curry fakes out Steve Kerr and knocks dow...
236561    [Highlights] All the plays from the insane fin...
236563    Heissler Guillent from Steph Curry range in Ve...
236564    Iman Shumpert and J.R. Smith with them double ...
236580     Highlights of Barkley's 48/10/15 WCF game (1993)
236583    Uncle Drew teaches an Youngblood the meaning o...
236592    Charles Barkley &amp; Muggsy Bogues prank J.R ...
236611    Giannis Antetokounmpo Full Highlights Bucks vs...
236620    iman shumpert and JR smith baked after winning...
236621              iman shumpert and JR smith baked on GMA
236630                           Doncic 

In [31]:
df['title'][df['type'] == 'gamethread']

11        GAME THREAD: Dallas Mavericks (17-18) @ Oklaho...
14        GAME THREAD: Memphis Grizzlies (18-17) @ Houst...
25        GAME THREAD: Minnesota Timberwolves (17-19) @ ...
75        GAME THREAD: Golden State Warriors (24-13) @ P...
586       GAME THREAD: Utah Jazz (18-19) @ Toronto Rapto...
                                ...                        
236517              GAME THREAD: Russia v Nigeria (Group B)
236556           GAME THREAD: Philippines v Italy (Group D)
236566         GAME THREAD: Cote d'Ivoire v China (Group A)
236568             GAME THREAD: Argentina v Korea (Group B)
236569               GAME THREAD: Spain v Tunisia (Group C)
Name: title, Length: 1015, dtype: object

#### We should probably take '[Highlight] and 'GAME THREAD' out of the post titles as to not make things too easy to classify


In [33]:
df = pipe.remove_type_tags_in_title(df)

In [34]:
df['title'][df['type'] == 'highlights'].tail(30)

236534    Jokic with the amazing tap pass to Marjanovic ...
236541    Steve Francise Mic'd Up Highlights 2001 Rookie...
236542               Josh Okogie hits the contested triple!
236545    Rod Strickland with a smooth behind-the-back s...
236546                           Jokic highlights vs Angola
236548    Josh Okogie with the beautiful feed to Chimezi...
236552    Dell Curry fakes out Steve Kerr and knocks dow...
236561    All the plays from the insane final minute in ...
236563    Heissler Guillent from Steph Curry range in Ve...
236564    Iman Shumpert and J.R. Smith with them double ...
236580     Highlights of Barkley's 48/10/15 WCF game (1993)
236583    Uncle Drew teaches an Youngblood the meaning o...
236592    Charles Barkley &amp; Muggsy Bogues prank J.R ...
236611    Giannis Antetokounmpo Full Highlights Bucks vs...
236620    iman shumpert and JR smith baked after winning...
236621              iman shumpert and JR smith baked on GMA
236630                           Doncic 

In [35]:
df['title'][df['type'] == 'gamethread']

11        Dallas Mavericks (17-18) @ Oklahoma City Thund...
14        Memphis Grizzlies (18-17) @ Houston Rockets (2...
25        Minnesota Timberwolves (17-19) @ New Orleans P...
75        Golden State Warriors (24-13) @ Phoenix Suns (...
586       Utah Jazz (18-19) @ Toronto Raptors (27-11) - ...
                                ...                        
236517                           Russia v Nigeria (Group B)
236556                        Philippines v Italy (Group D)
236566                      Cote d'Ivoire v China (Group A)
236568                          Argentina v Korea (Group B)
236569                            Spain v Tunisia (Group C)
Name: title, Length: 1015, dtype: object

In [38]:
save_name = 'posts_2019_all_cleaned.csv'
pipe.save_csv(df, save_name)

Saving to:  ~/Galvanize/Projects/data/Capstone2/posts_2019_all_cleaned.csv
